<a href="https://colab.research.google.com/github/rafaelpivetta/tech-challenge-fase3/blob/main/Copy_of_Running_LLama3_Chapter3_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-cpp-python
!pip install huggingface_hub
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.90-cp310-cp310-linux_x86_64.whl size=3398635 sha256=35c6b5278f4f48e5d67958f508030faecc97c348ffc8b3505b74d9d463ee5e8b
  Stored in directory: /root/.cache/pip/wheels/3d/67/02/f950031435db4a5a02e6269f6adb6703bf1631c3616380f3c6
Successfully built llama-cpp-python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
from llama_cpp import Llama
from datasets import load_dataset, Dataset

from datasets import load_dataset

ds = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset",
                  split="train")
print(ds.column_names)
print(ds.shape)

['flags', 'instruction', 'category', 'intent', 'response']
(26872, 5)


In [ ]:
first_thousand_points = ds[:1000]
ds = Dataset.from_dict(first_thousand_points)

In [ ]:
import pprint
pprint.pprint(ds[0])

{'category': 'ORDER',
 'flags': 'B',
 'instruction': 'question about cancelling order {{Order Number}}',
 'intent': 'cancel_order',
 'response': "I've understood you have a question regarding canceling order "
             "{{Order Number}}, and I'm here to provide you with the "
             'information you need. Please go ahead and ask your question, and '
             "I'll do my best to assist you."}


In [ ]:
def merge_example(row):
  row['conversation'] = f"Query: {row['instruction']}\nResponse: {row['response']}"
  return row

# Call the ds map method to apply our preprocessing function to all rows
ds = ds.map(merge_example) # Aplica a função para cada linha
print(ds[0]['conversation'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Query: question about cancelling order {{Order Number}}
Response: I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Maykeye/TinnyLlama-v0"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

training_arguments = TrainingArguments(
    per_device_train_batch_size=1,
    learning_rate=2e-4,
    max_grand_norm=0.3,
    max_steps=200,
    gradient_accumulation_steps=2,
    save_steps=2)

SFTTrainer(model=model,
           tokenizer=tokenizer,
           args=training_arguments,
           train_dataset=ds,
           dataset_text_field='conversation',
           max_sequence_length=250,
           args=training_arguments).train()




In [ ]:
import evaluate

rouge = evaluate.load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", ""]
results = rouge.compute(predictions=predictions, references=references)
print(results)

def generate_predictions_and_references(ds):
  predictions = []
  references = []
  for row in ds:
    inputs = tokenizer.encode(row["instruction"], return_tensors="pt")
    outputs = model.generate(inputs)
    decoded_output = tokenizer.decode(outputs[0, inputs.shape[1]:], skip_special_tokens=True)
    references += [row["response"]]
    predictions += [decoded_output]
  return references, predictions

references, predictions = generate_predictions_and_references(ds)
results = rouge.compute(predictions=predictions, references=references)
print(results)